In [1]:
import requests
import time
import os
import sys
import re
import io
import json
import csv
import openpyxl
import pandas as pd
import numpy as np
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Komoran, Okt
from nltk import Text
from pykospacing import spacing
import kss

In [2]:
browser = Chrome()

titles = []
links = []

date = input("시작날짜 입력(eg.20210920):")

#링크, 제목
for i in range(0,5):
    
    for j in range(1,3):
        browser.get("https://pann.nate.com/talk/ranking/d?stdt=" + str(date) + "&page=" + str(j))
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        first_list = soup.find('div', {'class': 'cntList'}).findAll('li')

    for li in first_list: 
        f_link = li.findAll('a')
        for a in f_link:
            real_link = 'https://pann.nate.com' + a.get('href') 
        links.append(real_link)
    
    for li in first_list:
        f_title = li.findAll('dl')
        for dl in f_title:
            t = dl.find('a')
            real_title = t.get('title')
            real_title = spacing(real_title)
            real_title = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', real_title)
            real_title = re.sub('[\xa0|0xed]', '', real_title)
            real_title = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]','',real_title)
        titles.append(real_title)
        
    date = int(date) + 1

#print({'제목': titles, 'url': links}, sep='\n')

시작날짜 입력(eg.20210920):20210920


In [3]:
#본문
txt = []
    
for i in links:
    try: 
        res = requests.get(i)
        res.raise_for_status()
        res.encoding = None            
        html2 = res.text
        
        soup = BeautifulSoup(html2, 'html.parser')
        contentArea = soup.find("div", {"class" : "viewarea"})            
        parags = contentArea.findAll("div", {"id" : "contentArea"})

        content = ""

        for parag in parags:
            content += parag.text
        content = re.sub('[ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', '', content)
        content = re.sub('[&nbsp;|\n|\t|\r]', '', content)
        content = re.sub('[\xa0]', '', content)
        content = re.sub('[-=+_★♥♡,#/\?:╋^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》—;]', '', content)
        content = re.sub('[^ ㄱ-ㅣ가-힣]','',content)
        #content = spacing(content)
        
        txt.append(content)
             
    except HTTPError as e:
        txt.append('')
    except URLError as e:
        txt.append('')
    except AttributeError as e:
        txt.append('')
        
#print(txt)

In [4]:
#형태소 분석
from konlpy.tag import Okt

okt = Okt() 

title_morphs = []
txt_morphs = []
    
for i in titles:
    title_morphs.append(okt.morphs(i))
    
for i in txt:
    txt_morphs.append(okt.morphs(i))
    

nate_dict = {
    '제목' : titles,
    '본문' : txt
}

df = pd.DataFrame(nate_dict) 
df.to_csv('natepann.csv', index=False, encoding="utf-8-sig")

  
morphs_dict = {
    '제목 형태소' : title_morphs,
    '본문 형태소' : txt_morphs

}


df2 = pd.DataFrame(morphs_dict)
df2.to_csv('natepann_Morphs.csv', index=False, encoding="utf-8-sig")

browser.close()

In [5]:
df.to_csv('natepann.txt', sep = '\t', index = False)

In [6]:
df = pd.read_csv('new.csv')

In [7]:
df.to_csv('new.txt', sep = '\t', index = False)

In [ ]:
df.to_csv('natepann_Morphs.txt', sep = '\t', index = False)